In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "qwiklabs-gcp-03-56464b18fd09"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [4]:
model = GenerativeModel("gemini-1.5-flash")

In [5]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [6]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried bouquets, playing on various themes:

**Evoking Timelessness and Durability:**

* Everlasting Blooms
* The Dried Flower Garden
* Timeless Bouquet
* Bloom & Preserve
* The Eternal Petal

**Highlighting the Craft and Uniqueness:**

* The Dried Flower Atelier
* The Botanical Studio
* The Dried Flower Collective
* Whispers of Wildflowers
* The Dried Flower Artistry

**Focusing on Nature and Sustainability:**

* The Wildflower Co.
* Earth & Bloom
* The Botanical Preserve
* Sun-Kissed Blooms
* Nature's Tapestry

**Short and Catchy:**

* Petal & Twig
* The Bloomery
* Dried & Delicate
* Bloom & Co.
* The Flower Bar

**Bonus:**

* [Your Name]'s Dried Flowers
* [City/Neighborhood] Dried Flowers

**Tips for Choosing a Name:**

* **Consider your target audience:** Who are you trying to attract?
* **Make it memorable:** A name that is easy to remember and pronounce is key.
* **Check for availability:** Make sure the name is not already tak

✅ Recommended. The prompt below is to the point and concise.

In [7]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Catchy & Creative:

* **Bloom Eternal**
* **Dried & Delightful**
* **Forever in Bloom**
* **The Dusty Bouquet**
* **Whispers of Wildflowers**
* **Sun Kissed Blooms**
* **Paper Petal Co.**
* **Botanical Reverie**
* **The Dried Flower Studio**
* **Everlasting Blooms**

## Elegant & Romantic:

* **The Timeless Bloom**
* **Whispers of Lavender**
* **The Dried Garden**
* **Bloom & Beyond**
* **The Botanical Muse**
* **Serene Blooms**
* **The Flower Alchemist**
* **The Dried Florist**
* **The Vintage Bouquet**
* **Ethereal Blooms**

## Playful & Unique:

* **Blooming Memories**
* **The Flower Fairy**
* **The Dried Bouquet Bar**
* **Petal & Dust**
* **The Floral Revival**
* **The Dusty Rose**
* **Blooming Nostalgia**
* **The Petal Press**
* **The Dried & Darling**
* **The Bloom Whisperer**

**Pro Tip:**

* Consider your target audience and the overall vibe you want to create.
* Keep the name short, memorable, and easy to pronounce.
* Check for available domain names and social media handle

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [8]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known place in the universe where life exists. Here's a glimpse into our incredible home:

**A Blue Marble:**

* **Surface:** Mostly covered in water (71%), with continents and islands making up the remaining landmass. 
* **Atmosphere:** A protective layer of gases, mainly nitrogen and oxygen, that keeps us warm and shields us from harmful radiation.
* **Unique Features:** Earth has a dynamic atmosphere with weather patterns, a diverse range of ecosystems, and active geological processes like volcanoes and earthquakes.

**Life Abundant:**

* **Biodiversity:** Earth is home to millions of species, from microscopic bacteria to giant whales.
* **Ecosystems:** From rainforests to deserts, oceans to mountains, Earth's diverse ecosystems support a complex web of life.
* **Humanity:** Our species, *Homo sapiens*, has evolved and thrived on Earth, shaping its landscape and culture.

**A Planet in Motion:**

* **Rota

✅ Recommended. The prompt below is specific and well-defined.

In [9]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What makes Earth unique?

Earth is a truly remarkable planet, and while other planets share some characteristics, Earth is unique in its combination of factors that make it the only known place harboring life as we know it. Here's a list of Earth's unique qualities:

**1. Liquid water:** Earth is the only planet in our solar system with liquid water on its surface. This is crucial for life, as water acts as a solvent, facilitates chemical reactions, and helps regulate temperature.

**2. Plate tectonics:** Earth's unique geological processes, driven by plate tectonics, create diverse landscapes, volcanic activity, and the recycling of Earth's crust. This dynamism helps regulate the climate and provides essential nutrients for life.

**3. Stable atmosphere:** Earth's atmosphere, rich in nitrogen and oxygen, provides a protective layer from harmful solar radiation and helps regulate temperature. The presence of oxygen is crucial for respiration and allows for the development of complex

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [10]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's break down these two questions:

**Boiling Water**

The "best" method depends on your needs and resources:

* **Stovetop:** This is the most common and versatile method. Use a pot with a wide base and a lid for faster heating. 
* **Electric Kettle:**  Fast and efficient, especially for small volumes.  
* **Microwave:**  Quick for small quantities, but may not boil evenly.
* **Campfire:**  A classic method, but requires fuel and time. 

**Why the Sky is Blue**

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's the simplified explanation:

* **Sunlight is a mix of colors:**  White sunlight is actually made up of all the colors of the rainbow.
* **Air molecules scatter light:**  When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (nitrogen and oxygen).
* **Shorter wavelengths scatter more:** Blue light has a shorter wavelength than other colors, so it scatters more readily off these molecules.
* **Blue light reaches our eyes

✅ Recommended. The prompts below asks one task a time.

In [11]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There's no single "best" method, as it depends on your needs and resources. Here's a breakdown of the most common methods and their pros and cons:

**1. Stovetop:**

* **Pros:** Widely accessible, fast, and allows for precise temperature control.
* **Cons:** Requires attention, can be inefficient if using a large pot, and may require a longer boil time for larger quantities.

**2. Electric Kettle:**

* **Pros:** Fast, efficient, and convenient. Many models have automatic shut-off features and temperature control options.
* **Cons:** Can be expensive, limited capacity, and not as versatile for other cooking tasks.

**3. Microwave:**

* **Pros:** Super fast, convenient, and energy-efficient for small quantities.
* **Cons:** Not ideal for large quantities, can be uneven heating, and some microwaves lack sufficient power.

**4. Camping Stove:**

* **Pros:** Portable and versatile for outdoor use.
* **Cons:** Requires fuel, can be slower than stovetop or electric kettle, and requires carefu

In [12]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky is blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown:

* **Sunlight is made of all colors:**  The sun emits light containing all colors of the rainbow. 
* **Air molecules scatter light:**  When sunlight enters the Earth's atmosphere, it interacts with tiny particles like nitrogen and oxygen molecules. These molecules are much smaller than the wavelengths of visible light.
* **Blue light scatters more:**  Shorter wavelengths of light, like blue and violet, are scattered more effectively by these small particles. This scattering happens in all directions.
* **We see the scattered blue light:**  As a result, we see the scattered blue light coming from all directions in the sky, making the sky appear blue.

**Why isn't the sky violet?** 

While violet light is scattered even more than blue, our eyes are more sensitive to blue light. This is why we perceive the sky as blue rather than violet.

**Other factors influencing sky color:**

* **Time of day:**  Th

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [19]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To know what day it is today, please check a calendar or a timekeeping device. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [22]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan offers a wealth of sightseeing opportunities! To give you the best recommendation, I need a bit more information. 

What are you most interested in seeing? 

* **Art and History:**  The Duomo, Sforza Castle, and the Pinacoteca di Brera are must-sees.
* **Fashion and Design:**  Visit the Galleria Vittorio Emanuele II, the Quadrilatero della Moda (fashion district), and the Museo del Novecento.
* **Culture and Entertainment:**  Catch a performance at Teatro alla Scala or explore the vibrant Navigli canals.

Tell me what you're looking for, and I can point you to the perfect spot in Milan! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [24]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help with travel planning, not personal decisions like what to eat. 

Do you have any questions about restaurants in Milan, or maybe you're looking for some local food recommendations? I'd be happy to help with that! 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [20]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

Let's find a programming activity that'll help you level up! To give you the best suggestion, tell me a bit about yourself:

**1. What programming languages are you familiar with?** (e.g., Python, Java, JavaScript, C++, etc.)
**2. What are you interested in?** (e.g., web development, game development, data science, machine learning, mobile app development, etc.)
**3. What's your current skill level?** (e.g., beginner, intermediate, advanced)

Once you answer these questions, I can recommend a project that's both fun and challenging for you.

Here are some ideas to get you started:

**For beginners:**

* **Build a simple text-based adventure game:** This helps you practice with variables, conditionals, loops, and input/output.
* **Create a program that generates a random password:** You'll learn about string manipulation, random number generation, and potentially some basic cryptography concepts.
* **Design a basic calculator:** This is a good way to solidify your understanding of opera

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [21]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

As a high school student, learning to code is a fantastic pursuit! All three languages have their uses, but here's a breakdown to help you choose:

**Python:**

* **Pros:**  
    * Beginner-friendly syntax, making it easier to learn.
    * Widely used in data science, machine learning, web development, and scripting.
    * Large and active community, meaning lots of resources and help available.
* **Cons:**
    * Can be slower than some other languages.

**JavaScript:**

* **Pros:**
    * The language of the web, essential for interactive websites and web applications.
    * High demand in the job market.
    * Can also be used for mobile app development (with frameworks).
* **Cons:**
    * Can be more complex to learn than Python. 

**Fortran:**

* **Pros:**
    * Traditionally used for scientific computing and high-performance applications.
    * Still relevant in certain fields like weather forecasting, physics simulations, and engineering.
* **Cons:**
    * Less popular than Python

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [25]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [26]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [27]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 

Here's why:

* **"Surprised me"** implies a positive outcome.
* **"Original"** is a positive attribute.
* **"Not the same old recycled stuff"** further reinforces the positive sentiment.
* **"Watch it - you will not regret it"**  is a strong recommendation, indicating enjoyment. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.